In [40]:
import sys
sys.path.insert(0,"/work/pip")

from __future__ import absolute_import
from __future__ import print_function

import numpy as np
import argparse
import os
import imp
import re

from mimic3models.in_hospital_mortality import utils
from mimic3benchmark.readers import Reader, InHospitalMortalityReader

from mimic3models.preprocessing import Discretizer, Normalizer, DiscretizerContinuous

In [11]:
class PatientEmbeddingReader(Reader):
    def __init__(self, dataset_dir, listfile=None, period_length=48.0):
        """ Reader for in-hospital moratality prediction task.

        :param dataset_dir:   Directory where timeseries files are stored.
        :param listfile:      Path to a listfile. If this parameter is left `None` then
                              `dataset_dir/listfile.csv` will be used.
        :param period_length: Double the Length of the period (in hours) from which the embedding is created.
        """
        Reader.__init__(self, dataset_dir, listfile)
        self._data = [line.split(',') for line in self._data]
        self._data = [(x, int(wt), int(w)) for (x, wt, w) in self._data]
        self._period_length = period_length
        self._input_dim = None
        
    def _read_timeseries(self, ts_filename):
        ret = []
        with open(os.path.join(self._dataset_dir, ts_filename), "r") as tsfile:
            header = tsfile.readline().strip().split(',')
            assert header[0] == "Hours"
            for line in tsfile:
                mas = line.strip().split(',')
                ret.append(np.array(mas))
            return (np.stack(ret), header)
    
    def get_input_dim(self):
        if self._input_dim is None:
            name, norm, end_time = self._data[0]
            (X, header) = self._read_timeseries(name)
            self._input_dim = X.shape[1] - 1
        return self._input_dim  
        
    def read_example(self, index):
        """ Reads the example with given index.

        :param index: Index of the line of the listfile to read (counting starts from 0).
        :return: Dictionary with the following keys:
            X : np.array
                2D array containing all events. Each row corresponds to a moment.
                First column is the time and other columns correspond to different
                variables.
            t : float
                Length of the data in hours. Note, in general, it is not equal to the
                timestamp of last event.
            y : int (0 or 1)
                In-hospital mortality.
            header : array of strings
                Names of the columns. The ordering of the columns is always the same.
            name: Name of the sample.
        """
        if index < 0 or index >= len(self._data):
            raise ValueError("Index must be from 0 (inclusive) to number of lines (exclusive).")
            
        name, norm, end_time = self._data[index]
        end_time += 48
        t = self._period_length
        (X, header) = self._read_timeseries(name)

        return {"X": X,
                "t": t,
                "norm": norm,
                "end_time": end_time,
                "header": header,
                "name": name}

In [12]:
# Data Directories
data_basedir = '/work/MIMIC/processed_data'
data_patient_embedding = os.path.join(data_basedir, 'patient_embeddings_test')

In [50]:
train_reader = PatientEmbeddingReader(dataset_dir=os.path.join(data_patient_embedding, 'train'), 
                                      listfile=os.path.join(data_patient_embedding, 'train', 'listfile_visit.csv'), 
                                      period_length=48.0)

In [14]:
discretizer = DiscretizerContinuous(timestep=1.0,
                                    store_masks=False,
                                    impute_strategy='previous',
                                    start_time='zero')

In [15]:
discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1]
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

In [16]:
cont_channels

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [17]:
discretizer.transform(train_reader.read_example(0)["X"])[0][1].shape

(17,)

In [18]:
discretizer_header

['Capillary refill rate',
 'Diastolic blood pressure',
 'Fraction inspired oxygen',
 'Glascow coma scale eye opening',
 'Glascow coma scale motor response',
 'Glascow coma scale total',
 'Glascow coma scale verbal response',
 'Glucose',
 'Heart Rate',
 'Height',
 'Mean blood pressure',
 'Oxygen saturation',
 'Respiratory rate',
 'Systolic blood pressure',
 'Temperature',
 'Weight',
 'pH']

In [19]:
normalizer = Normalizer(fields=cont_channels)
normalizer_state = 'ptemb_ts1.0.input_str:previous.start_time:zero.normalizer'
normalizer_state = os.path.join("/work/mimic3-benchmarks/mimic3models/patient_embedding", normalizer_state)
normalizer.load_params(normalizer_state)

In [20]:
normalizer._stds.shape

(17,)

In [21]:
train_reader.read_example(0)["X"]

array([['0.3030555555555556', '', '67.0', ..., '36.72222137451172', '',
        ''],
       ['0.5530555555555555', '', '59.0', ..., '', '', ''],
       ['1.5530555555555556', '', '62.0', ..., '', '', ''],
       ...,
       ['53.55305555555556', '', '68.0', ..., '', '', ''],
       ['54.55305555555556', '', '53.0', ..., '', '', ''],
       ['55.55305555555556', '', '58.0', ..., '38.16669845581055', '',
        '']], dtype='<U18')

In [51]:
import torch
from torch.utils.data import Dataset, DataLoader
import copy
from mimic3models.pytorch_models.embedding.model.utils.TRANS import subsequent_mask

#Pytorch Wrapper for load_data --> Feeds of PatientEmbeddingReader
class PatientEmbeddingDataset(Dataset):
    def __init__(self, reader, discretizer, embed_method = 'TRANS', normalizer=None, mask_percent=.15, return_name=False):
        self.reader = reader
        self.discretizer = discretizer
        self.normalizer = normalizer
        self.mask_percent = mask_percent
        self.return_name = return_name
        self.embed_method = embed_method
        self.n_visits = self.reader.get_number_of_visits()
        self.input_dim = self.reader.get_input_dim()
        self.seq_length = int(self.reader._period_length/2)
    
    def __len__(self):
        return self.reader.get_number_of_examples()
    
    def get_number_of_visits(self):
        return self.n_visits
    
    def get_input_dim(self):
        return self.input_dim
    
    def get_seq_length(self):
        return self.seq_length
    
    def __getitem__(self, index):
        ret = self.reader.read_example(index)
        
        X = ret["X"]
        t = ret["t"]
        end = ret["end_time"]
        name = ret["name"]
        norm = np.array(ret["norm"])
        
        X = self.discretizer.transform(X, end=end)[0][-int(t):]
        if self.normalizer is not None:
            X = self.normalizer.transform(X)
        
        src = np.array(X[-int(t):-int(t/2)])
        
        if self.embed_method is 'TRANS':
            tgt = np.array(X[-int(t/2):])

            mask = np.zeros(int(t/2))
            n_masks = round(self.mask_percent*int(t/2))
            mask_ids = np.random.permutation(int(t/2))[:n_masks]
            mask[mask_ids] = 1

            src_masked = copy.deepcopy(src)
            src_masked[(mask==1), :] = 0

            tgt_input = np.vstack((src[-1,:], tgt[:-1,:]))
            tgt_mask = subsequent_mask(tgt_input.shape[0]).squeeze(0)
            
            data = {'src_masked':src_masked, 'src':src, 
                    'tgt_input':tgt_input, 'tgt':tgt, 'tgt_mask': tgt_mask,
                    'norm':norm, 'mask':mask}
        else:
            src = src.flatten()
            data = {'src':src, 'tgt':src, 'norm':norm}
        
        if self.return_name:
            data['name'] = name
        
        return data 

In [52]:
PEmbDataset = PatientEmbeddingDataset(reader=train_reader, discretizer=discretizer, 
                                      normalizer=normalizer, return_name=True, embed_method = 'PCA')


In [53]:
training_generator = DataLoader(PEmbDataset, batch_size=512, shuffle=True, num_workers=12)

In [54]:
PEmbDataset.seq_length

24

In [55]:
for i, data in enumerate(training_generator):
    if i == 1:
        break
    print(data['src'].shape)

torch.Size([61, 408])


In [59]:
data['src'].numpy()

array([[-0.04793494, -0.00520967,  0.56633609, ..., -0.26012714,
        -1.68070967,  0.1079599 ],
       [-0.04793494,  0.07158918, -0.45483597, ...,  0.12078707,
        -0.98233921,  0.05381554],
       [-0.04793494, -0.04011823,  0.56633609, ..., -0.08237396,
         1.52602064,  0.08539975],
       ...,
       [-0.04793494, -0.02964566, -0.45483597, ..., -0.00618859,
        -0.03748488, -0.03191304],
       [-0.04793494,  0.0052629 , -0.45483597, ...,  0.12077254,
        -0.07343042,  0.07637569],
       [-0.04793494, -0.00870052, -0.45483597, ...,  0.31884045,
        -0.03748488,  0.08088772]])

In [61]:
from sklearn.decomposition import IncrementalPCA

In [7]:
x = 'TRAN'
if x not in ['TRANS', 'DAE', 'PCA', 'RAW']:
    print('nope')

nope


In [37]:
data = PEmbDataset[0]
data['tgt_mask']

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1